# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mermoz boabab,14.7065,-17.4758,27.88,94,100,0.00,SN,1697430838
1,1,waitangi,-43.9535,-176.5597,12.68,71,53,2.68,NZ,1697430409
2,2,amahai,-3.3333,128.9167,28.34,77,29,2.94,ID,1697430838
3,3,flin flon,54.7682,-101.8650,6.47,74,65,2.42,CA,1697430451
4,4,anadyr,64.7500,177.4833,1.28,100,75,2.00,RU,1697430415


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_data_df['Lng']

0      -17.4758
1     -176.5597
2      128.9167
3     -101.8650
4      177.4833
         ...   
580    -75.4557
581    133.2333
582     57.5281
583    -50.0500
584    -53.2648
Name: Lng, Length: 585, dtype: float64

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5
)
# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria - temp between 15 and 25 celsius, mostly sunny, not very humid
sunny_moderate_cities = city_data_df[(city_data_df['Max Temp'] >= 15) & (city_data_df['Max Temp'] <= 25) & (city_data_df['Cloudiness'] < 50) & (city_data_df['Humidity'] < 75)]

# Drop any rows with null values
sunny_moderate_cities = sunny_moderate_cities.dropna()

# Display sample data
sunny_moderate_cities.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,awjilah,29.1081,21.2869,19.22,69,0,1.54,LY,1697430839
36,36,kandira,41.0736,30.1746,18.97,70,10,3.60,TR,1697430841
76,76,xingcheng,40.6167,120.7167,16.91,19,0,2.54,CN,1697430846
107,107,texas city,29.3838,-94.9027,17.99,61,0,8.75,US,1697430806
108,108,jinan-gun,35.7917,127.4253,17.26,45,0,3.99,KR,1697430850
110,110,yangliuqing,39.1375,116.9997,20.97,29,0,2.37,CN,1697430850
126,126,christchurch,-43.5333,172.6333,19.11,32,40,5.66,NZ,1697430661
157,157,broken hill,-31.9500,141.4333,21.11,25,1,9.05,AU,1697430487
163,163,port lincoln,-34.7333,135.8667,15.14,53,37,6.69,AU,1697430434
166,166,al kharijah,25.4514,30.5464,21.22,40,0,3.07,EG,1697430857


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = sunny_moderate_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
15,awjilah,LY,29.1081,21.2869,69,
36,kandira,TR,41.0736,30.1746,70,
76,xingcheng,CN,40.6167,120.7167,19,
107,texas city,US,29.3838,-94.9027,61,
108,jinan-gun,KR,35.7917,127.4253,45,
110,yangliuqing,CN,39.1375,116.9997,29,
126,christchurch,NZ,-43.5333,172.6333,32,
157,broken hill,AU,-31.9500,141.4333,25,
163,port lincoln,AU,-34.7333,135.8667,53,
166,al kharijah,EG,25.4514,30.5464,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1

params = {
    'categories':categories,
    'apiKey':geoapify_key,
    'limit':limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
awjilah - nearest hotel: No hotel found
kandira - nearest hotel: Kandıra Öğretmenevi
xingcheng - nearest hotel: No hotel found
texas city - nearest hotel: Comfort Inn & Suites Texas City
jinan-gun - nearest hotel: 호텔 홍삼빌
yangliuqing - nearest hotel: IU Inn
christchurch - nearest hotel: Ibis Hotel Christchurch
broken hill - nearest hotel: Ibis Styles
port lincoln - nearest hotel: Boston Hotel
al kharijah - nearest hotel: Kharga Hotel
foca - nearest hotel: Villa Dedem
san patricio - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
berre-l'etang - nearest hotel: Chez Babette
metlili chaamba - nearest hotel: No hotel found
naze - nearest hotel: ビジネス旅館畠山
rudsar - nearest hotel: هتل بهار
yingchuan - nearest hotel: No hotel found
fedorovskiy - nearest hotel: Маяк
idri - nearest hotel: No hotel found
suozhen - nearest hotel: No hotel found
mek'ele - nearest hotel: Union Hotel
illizi - nearest hotel: بوناقة
eilat - nearest hotel: Melony Club
hasaki - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
15,awjilah,LY,29.1081,21.2869,69,No hotel found
36,kandira,TR,41.0736,30.1746,70,Kandıra Öğretmenevi
76,xingcheng,CN,40.6167,120.7167,19,No hotel found
107,texas city,US,29.3838,-94.9027,61,Comfort Inn & Suites Texas City
108,jinan-gun,KR,35.7917,127.4253,45,호텔 홍삼빌
110,yangliuqing,CN,39.1375,116.9997,29,IU Inn
126,christchurch,NZ,-43.5333,172.6333,32,Ibis Hotel Christchurch
157,broken hill,AU,-31.9500,141.4333,25,Ibis Styles
163,port lincoln,AU,-34.7333,135.8667,53,Boston Hotel
166,al kharijah,EG,25.4514,30.5464,40,Kharga Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.8,
    color = "City",
    alpha = 0.5,
    hover_cols = ['Hotel Name', 'Country']
)
# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)